## 1. Setup and Imports

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("Basic imports loaded successfully!")

In [ ]:
# ML Libraries
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("Scikit-learn imports loaded successfully!")

In [ ]:
# Deep Learning Libraries (PyTorch)
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader, TensorDataset
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA device: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("PyTorch not installed. Install with: pip install torch")

## 2. F3 - Time Series Forecasting Models

The forecasting service uses ML models to predict:
- Water levels (1-72 hours ahead)
- Rainfall patterns
- Flood/drought risk assessment

### 2.1 Understanding the Current Forecasting System

In [ ]:
# Simulate historical data similar to the forecasting service
import time
import random

def generate_water_level_data(days=30, hourly=True):
    """Generate simulated water level data with seasonal patterns."""
    periods = days * 24 if hourly else days
    base_time = datetime.now() - timedelta(days=days)
    
    data = []
    for i in range(periods):
        if hourly:
            timestamp = base_time + timedelta(hours=i)
        else:
            timestamp = base_time + timedelta(days=i)
        
        # Seasonal pattern
        day_of_year = timestamp.timetuple().tm_yday
        seasonal_factor = 0.5 + 0.5 * np.sin(2 * np.pi * day_of_year / 365)
        
        # Water level (0-100% of capacity)
        base_level = 60 + 20 * seasonal_factor
        water_level = max(10, min(95, base_level + random.uniform(-10, 10)))
        
        # Rainfall (mm per hour/day)
        rainfall_prob = 0.3 if seasonal_factor > 0.6 else 0.1
        rainfall = random.uniform(0, 15) if random.random() < rainfall_prob else 0
        
        # Dam gate opening
        gate_opening = min(80, max(0, water_level - 50 + random.uniform(-10, 10)))
        
        data.append({
            'timestamp': timestamp,
            'water_level_percent': round(water_level, 2),
            'rainfall_mm': round(rainfall, 2),
            'gate_opening_percent': round(gate_opening, 2)
        })
    
    return pd.DataFrame(data)

# Generate sample data
df_water = generate_water_level_data(days=60)
print(f"Generated {len(df_water)} hours of water level data")
df_water.head()

In [ ]:
# Visualize the water level data
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Water Level
axes[0].plot(df_water['timestamp'], df_water['water_level_percent'], 'b-', alpha=0.7)
axes[0].set_ylabel('Water Level (%)')
axes[0].set_title('Water Level Over Time')
axes[0].axhline(y=85, color='r', linestyle='--', label='Flood Warning')
axes[0].axhline(y=20, color='orange', linestyle='--', label='Drought Warning')
axes[0].legend()

# Rainfall
axes[1].bar(df_water['timestamp'], df_water['rainfall_mm'], color='cyan', alpha=0.7)
axes[1].set_ylabel('Rainfall (mm)')
axes[1].set_title('Rainfall Over Time')

# Gate Opening
axes[2].plot(df_water['timestamp'], df_water['gate_opening_percent'], 'g-', alpha=0.7)
axes[2].set_ylabel('Gate Opening (%)')
axes[2].set_title('Dam Gate Opening Over Time')

plt.tight_layout()
plt.show()

### 2.2 LSTM Model for Water Level Forecasting

Long Short-Term Memory (LSTM) networks are ideal for time series forecasting because they can:
- Remember long-term dependencies
- Handle sequential data with varying time lags
- Learn complex temporal patterns

In [ ]:
# Define LSTM Model Architecture
class WaterLevelLSTM(nn.Module):
    """
    LSTM-based model for water level forecasting.
    
    Architecture:
    - Input: Sequence of [water_level, rainfall, gate_opening]
    - LSTM layers with dropout for regularization
    - Fully connected output layer
    """
    def __init__(self, input_size=3, hidden_size=64, num_layers=2, output_size=1, dropout=0.2):
        super(WaterLevelLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, output_size)
        )
    
    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # LSTM forward pass
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Take the last output
        out = self.fc(lstm_out[:, -1, :])
        return out

# Print model architecture
model = WaterLevelLSTM()
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Prepare data for LSTM training
def create_sequences(data, seq_length=24):
    """
    Create sequences for LSTM training.
    
    Args:
        data: DataFrame with features
        seq_length: Number of time steps to look back
    
    Returns:
        X: Input sequences (batch, seq_length, features)
        y: Target values (batch, 1)
    """
    features = ['water_level_percent', 'rainfall_mm', 'gate_opening_percent']
    target = 'water_level_percent'
    
    # Scale the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[features])
    
    X, y = [], []
    for i in range(seq_length, len(scaled_data)):
        X.append(scaled_data[i-seq_length:i])
        y.append(scaled_data[i, 0])  # Water level is first feature
    
    return np.array(X), np.array(y), scaler

# Create sequences
SEQ_LENGTH = 24  # Use 24 hours of history
X, y, scaler = create_sequences(df_water, SEQ_LENGTH)

print(f"Input shape: {X.shape} (samples, sequence_length, features)")
print(f"Output shape: {y.shape} (samples,)")

In [ ]:
# Train/test split
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# Training function
def train_model(model, train_loader, X_test, y_test, epochs=50, lr=0.001):
    """
    Train the LSTM model.
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_test)
            val_loss = criterion(val_outputs, y_test).item()
        
        train_losses.append(epoch_loss / len(train_loader))
        val_losses.append(val_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_loss:.4f}")
    
    return train_losses, val_losses

# Train the model
model = WaterLevelLSTM(input_size=3, hidden_size=64, num_layers=2)
train_losses, val_losses = train_model(model, train_loader, X_test_tensor, y_test_tensor, epochs=50)

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
axes[0].plot(train_losses, label='Training Loss')
axes[0].plot(val_losses, label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()

# Predictions vs Actual
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).numpy()

axes[1].plot(y_test[:100], label='Actual', alpha=0.7)
axes[1].plot(predictions[:100], label='Predicted', alpha=0.7)
axes[1].set_xlabel('Time Step')
axes[1].set_ylabel('Water Level (Normalized)')
axes[1].set_title('Predictions vs Actual (First 100 samples)')
axes[1].legend()

plt.tight_layout()
plt.show()

## 3. F4 - ACA-O Optimization Models

The ACA-O (Adaptive Crop & Area Optimization) service uses:
- **Fuzzy-TOPSIS**: Multi-criteria decision making for crop suitability
- **Yield Prediction Model**: Predicts crop yield based on field conditions
- **Price Prediction Model**: Forecasts market prices

### 3.1 Fuzzy-TOPSIS Suitability Scoring

In [ ]:
# Fuzzy-TOPSIS Implementation
class FuzzyTOPSIS:
    """
    Fuzzy TOPSIS for multi-criteria crop suitability ranking.
    
    TOPSIS ranks alternatives by their distance to:
    - Positive Ideal Solution (PIS): Best possible values
    - Negative Ideal Solution (NIS): Worst possible values
    """
    
    def __init__(self, criteria_weights=None):
        self.default_weights = {
            'soil_suitability': 0.25,
            'water_coverage_ratio': 0.25,
            'historical_yield': 0.20,
            'water_sensitivity': 0.15,
            'growth_duration': 0.15
        }
        self.weights = criteria_weights or self.default_weights
    
    def compute_scores(self, alternatives):
        """
        Compute suitability scores for crop alternatives.
        
        Args:
            alternatives: Dict of crop_id -> features dict
        
        Returns:
            Dict of crop_id -> suitability score (0-1)
        """
        scores = {}
        for crop_id, features in alternatives.items():
            score = self._compute_single_score(features)
            scores[crop_id] = score
        return scores
    
    def _compute_single_score(self, features):
        """Compute score for single crop using weighted criteria."""
        # Normalize features
        soil_suit = features.get('soil_suitability', 0.5)
        water_coverage = features.get('water_coverage_ratio', 0.5)
        
        # Historical yield normalized to max 10 t/ha
        hist_yield = features.get('historical_yield_t_ha', 3.0)
        hist_yield_norm = min(1.0, hist_yield / 10.0)
        
        # Water sensitivity (low=1.0, medium=0.6, high=0.3)
        water_sens = features.get('water_sensitivity', 'medium')
        water_sens_score = {'low': 1.0, 'medium': 0.6, 'high': 0.3}.get(water_sens, 0.6)
        
        # Growth duration (shorter is better, max 180 days)
        duration = features.get('growth_duration_days', 120)
        duration_score = max(0, 1 - (duration / 180))
        
        # Weighted sum
        score = (
            self.weights['soil_suitability'] * soil_suit +
            self.weights['water_coverage_ratio'] * water_coverage +
            self.weights['historical_yield'] * hist_yield_norm +
            self.weights['water_sensitivity'] * water_sens_score +
            self.weights['growth_duration'] * duration_score
        )
        
        return round(max(0.0, min(1.0, score)), 3)

# Example usage
topsis = FuzzyTOPSIS()

# Sample crop alternatives
crops = {
    'Rice': {
        'soil_suitability': 0.9,
        'water_coverage_ratio': 0.95,
        'historical_yield_t_ha': 5.5,
        'water_sensitivity': 'high',
        'growth_duration_days': 120
    },
    'Maize': {
        'soil_suitability': 0.85,
        'water_coverage_ratio': 0.7,
        'historical_yield_t_ha': 6.0,
        'water_sensitivity': 'medium',
        'growth_duration_days': 90
    },
    'Soybean': {
        'soil_suitability': 0.75,
        'water_coverage_ratio': 0.6,
        'historical_yield_t_ha': 3.5,
        'water_sensitivity': 'low',
        'growth_duration_days': 100
    },
    'Vegetables': {
        'soil_suitability': 0.8,
        'water_coverage_ratio': 0.85,
        'historical_yield_t_ha': 8.0,
        'water_sensitivity': 'high',
        'growth_duration_days': 60
    }
}

scores = topsis.compute_scores(crops)
ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)

print("Crop Suitability Ranking (Fuzzy-TOPSIS):")
print("-" * 40)
for rank, (crop, score) in enumerate(ranked, 1):
    print(f"{rank}. {crop}: {score:.3f}")

In [ ]:
# Visualize crop suitability comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of scores
crop_names = [c[0] for c in ranked]
crop_scores = [c[1] for c in ranked]
colors = plt.cm.RdYlGn(np.array(crop_scores))

axes[0].barh(crop_names, crop_scores, color=colors)
axes[0].set_xlabel('Suitability Score')
axes[0].set_title('Crop Suitability Scores')
axes[0].set_xlim(0, 1)

# Radar chart of criteria
categories = ['Soil', 'Water', 'Yield', 'W.Sensitivity', 'Duration']
N = len(categories)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

ax = axes[1]
ax = plt.subplot(122, polar=True)

for crop_name, features in list(crops.items())[:3]:  # Top 3 crops
    values = [
        features['soil_suitability'],
        features['water_coverage_ratio'],
        min(1.0, features['historical_yield_t_ha'] / 10),
        {'low': 1.0, 'medium': 0.6, 'high': 0.3}[features['water_sensitivity']],
        max(0, 1 - features['growth_duration_days'] / 180)
    ]
    values += values[:1]
    ax.plot(angles, values, linewidth=2, label=crop_name)
    ax.fill(angles, values, alpha=0.1)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_title('Crop Criteria Comparison')
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout()
plt.show()

### 3.2 Yield Prediction Model

The yield prediction model estimates crop yield based on:
- Soil characteristics (pH, EC, texture)
- Water availability
- Climate conditions
- Historical performance

In [ ]:
# Neural Network for Yield Prediction
class YieldPredictionMLP(nn.Module):
    """
    Multi-Layer Perceptron for crop yield prediction.
    
    Input features:
    - soil_suitability: Soil match score (0-1)
    - water_coverage_ratio: Water availability ratio (0-1)
    - soil_ph: Soil pH value (4-9)
    - soil_ec: Electrical conductivity (mS/cm)
    - season_avg_temp: Average temperature (°C)
    - season_rainfall_mm: Total rainfall (mm)
    - growth_duration_days: Crop growth period
    
    Output:
    - Predicted yield (tonnes/hectare)
    """
    
    def __init__(self, input_size=7, hidden_sizes=[64, 32, 16]):
        super(YieldPredictionMLP, self).__init__()
        
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(0.2)
            ])
            prev_size = hidden_size
        
        layers.append(nn.Linear(prev_size, 1))
        layers.append(nn.ReLU())  # Yield must be positive
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Create model
yield_model = YieldPredictionMLP()
print(yield_model)
print(f"\nTotal parameters: {sum(p.numel() for p in yield_model.parameters()):,}")

In [ ]:
# Generate synthetic yield data for demonstration
np.random.seed(42)
n_samples = 1000

# Generate features
yield_data = pd.DataFrame({
    'soil_suitability': np.random.uniform(0.3, 1.0, n_samples),
    'water_coverage_ratio': np.random.uniform(0.4, 1.0, n_samples),
    'soil_ph': np.random.uniform(5.0, 8.0, n_samples),
    'soil_ec': np.random.uniform(0.5, 2.5, n_samples),
    'season_avg_temp': np.random.uniform(20, 35, n_samples),
    'season_rainfall_mm': np.random.uniform(100, 500, n_samples),
    'growth_duration_days': np.random.uniform(60, 150, n_samples)
})

# Generate target (yield) with realistic relationship
yield_data['yield_t_ha'] = (
    3.0 +
    yield_data['soil_suitability'] * 4 +
    yield_data['water_coverage_ratio'] * 3 +
    (yield_data['soil_ph'] - 6.5).abs() * -0.5 +  # pH closer to neutral is better
    yield_data['season_rainfall_mm'] / 200 +
    np.random.normal(0, 0.5, n_samples)  # Random noise
).clip(0.5, 12.0)  # Realistic yield range

print(yield_data.describe())

In [ ]:
# Visualize yield data correlations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

features_to_plot = ['soil_suitability', 'water_coverage_ratio', 'soil_ph', 
                    'season_rainfall_mm', 'season_avg_temp', 'growth_duration_days']

for ax, feature in zip(axes.flatten(), features_to_plot):
    ax.scatter(yield_data[feature], yield_data['yield_t_ha'], alpha=0.5, s=10)
    ax.set_xlabel(feature)
    ax.set_ylabel('Yield (t/ha)')
    ax.set_title(f'Yield vs {feature}')

plt.tight_layout()
plt.show()

## 4. Model Evaluation Metrics

Key metrics for evaluating the models:
- **RMSE** (Root Mean Square Error): Penalizes large errors
- **MAE** (Mean Absolute Error): Average absolute difference
- **MAPE** (Mean Absolute Percentage Error): Percentage-based error
- **R²** (Coefficient of Determination): Explained variance

In [ ]:
def evaluate_model(y_true, y_pred, model_name="Model"):
    """
    Calculate and display model evaluation metrics.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n{model_name} Evaluation Metrics:")
    print("=" * 40)
    print(f"RMSE:  {rmse:.4f}")
    print(f"MAE:   {mae:.4f}")
    print(f"MAPE:  {mape:.2f}%")
    print(f"R²:    {r2:.4f}")
    
    return {'rmse': rmse, 'mae': mae, 'mape': mape, 'r2': r2}

# Example evaluation (using our LSTM predictions)
evaluate_model(y_test, predictions.flatten(), "Water Level LSTM")

## 5. Next Steps

To further develop the ML models:

1. **Data Collection**
   - Gather real sensor data from irrigation systems
   - Collect historical yield data from farms
   - Integrate satellite imagery for crop health

2. **Model Improvements**
   - Implement attention mechanisms for LSTM
   - Use ensemble methods for yield prediction
   - Add uncertainty quantification

3. **Production Deployment**
   - Export models using TorchScript or ONNX
   - Set up MLflow for experiment tracking
   - Create model versioning pipeline

In [ ]:
# Save model checkpoint example
import os

# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Save the trained LSTM model
torch.save({
    'model_state_dict': model.state_dict(),
    'model_config': {
        'input_size': 3,
        'hidden_size': 64,
        'num_layers': 2,
        'output_size': 1
    },
    'scaler': scaler,
    'training_info': {
        'epochs': 50,
        'final_train_loss': train_losses[-1],
        'final_val_loss': val_losses[-1]
    }
}, '../models/water_level_lstm.pth')

print("Model saved to ../models/water_level_lstm.pth")